In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
# setting path
sys.path.append("/content/drive/MyDrive/simclr")
sys.path.append("/content/drive/MyDrive/simclr/layers")
sys.path.append("/content/drive/MyDrive/simclr/models")
sys.path.append("/content/drive/MyDrive/simclr/loss")
sys.path.append("/content/drive/MyDrive/simclr/optim")
sys.path.append("/content/drive/MyDrive/simclr/dataloader")
sys.path.append("/content/drive/MyDrive/simclr/instance-discrim")
sys.path.append("/content/drive/MyDrive/simclr/dloader")

In [3]:
from simclr import SimClr
from ntxent import nt_xent_loss
from ntxentgit import SimCLR_Loss
from augment import TransformsSimCLR
from utils import *
from LARS import LARS
from downstream import DownStream
from dataloader import *
from ResNetCifar import ResNetCifar
from utilsInstance import *
from NCEAverage import *
from NCECriterion import *
from alias_multinomial import *
from Cifar10Instance import *
from downstreamnew import DownStreamNew

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pickle

In [5]:
shuffle = True
train_loader_ds, val_loader_ds = train_loader_dstream("CIFAR10",256,0.1,shuffle)

Files already downloaded and verified


In [6]:
model = SimClr('resnet50',64).cuda()
model.load_state_dict(torch.load("/content/drive/MyDrive/simclr/saved-models/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS"))

<All keys matched successfully>

In [7]:
model_ds = DownStreamNew(model.encoder,model.projector.layers[:2],2048,10).cuda()
optimizer = torch.optim.Adam(model_ds.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
epochs = 100
model_ds,train_loss_list,train_acc_list,val_loss_list,val_acc_list = train_ds(train_loader_ds, val_loader_ds, model_ds, criterion, optimizer,epochs,True,path="/content/drive/MyDrive/simclr/dstream-saved-models/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS")

Epoch: 0, step: 50/175, training_loss: 0.8859522938728333, training_acc: 0.7109375
Epoch: 0, step: 100/175, training_loss: 0.855906069278717, training_acc: 0.67578125
Epoch: 0, step: 150/175, training_loss: 0.6492679119110107, training_acc: 0.77734375
Epoch: 0 completed, average train loss: 0.8754637973649161, average training_acc: 0.7068080357142857, average validation loss: 0.7225865062914396, average validation_acc: 0.7442434210526315 time taken: 0.429303232828776 mins
Epoch: 1, step: 50/175, training_loss: 0.7807612419128418, training_acc: 0.69140625
Epoch: 1, step: 100/175, training_loss: 0.7598190903663635, training_acc: 0.7421875
Epoch: 1, step: 150/175, training_loss: 0.63387531042099, training_acc: 0.79296875
Epoch: 1 completed, average train loss: 0.7171259924343654, average training_acc: 0.74484375, average validation loss: 0.698281445001301, average validation_acc: 0.7557565789473685 time taken: 0.4258216381072998 mins
Epoch: 2, step: 50/175, training_loss: 0.77253425121307

In [8]:
with open("/content/drive/MyDrive/simclr/dstream-trainloss-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS", "wb") as fp:   #Pickling
    pickle.dump(train_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-trainacc-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS", "wb") as fp:   #Pickling
    pickle.dump(train_acc_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valloss-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS", "wb") as fp:   #Pickling
    pickle.dump(val_loss_list, fp)
with open("/content/drive/MyDrive/simclr/dstream-valacc-logs/CIFAR10-RES50-SIMCLR-BS128-PD64-LARS", "wb") as fp:   #Pickling
    pickle.dump(val_acc_list, fp)

In [9]:
test_loader = test_loader("CIFAR10",128)

Files already downloaded and verified


In [10]:
test_ds(model_ds,test_loader)

Test accuracy: 0.7847556089743589, time taken: 0.08288057247797648
